Top 10 películas Argentinas según vote_average que contengan el género drama con más de 100 reviews en ratings.csv (dar géneros, cantidad de ratings, título, y vote_average).

In [ ]:
#Realizamos las importanciones
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 57.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=1dcee90638f7e07aa006cd335c818b2b1487ba8a52e68c9e792d6cb1f33e24d4
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#Realizamos las importaciones para usar google drive y pyspark
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Descargamos movies.parquet
descarga = drive.CreateFile({'id':'1YKh46_KdgSC-6UZ0mttbOqVI6tXgly-F'})
descarga.GetContentFile('movies.parquet')

In [ ]:
#Descargamos ratings.parquet
descarga = drive.CreateFile({'id':'1fE0C3RJuvKs-bwEZeapVmjDDhNDr9npu'})
descarga.GetContentFile('ratings.parquet')

In [ ]:
spark = SparkSession.builder.getOrCreate() #Obtenemos una sesion de spark
sc = spark.sparkContext #Obtenemos el contexto

In [ ]:
#Creo los rdd con los que voy a trabajar
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('ratings.parquet',header=True,inferSchema=True)
ratingsRdd = df.rdd
df = sqlContext.read.parquet('movies.parquet',header=True,inferSchema=True)
moviesRdd = df.rdd 

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
#Primero filtro las peliculas que tengan mas de 100 reviews. Para eso lo que hago es un map quedandome (movieId,1), luego realizo un reduceByKey contando los ratings para cada pelicula y luego un filter
ratingsRdd = ratingsRdd.map(lambda x: (int(x.movieId),1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>100)
ratingsRdd.take(5) #Vemos que ratingsRdd me queda como (movieId,cantRatings)

[(110, 66512), (858, 57070), (1246, 25752), (1968, 26611), (2762, 49643)]

In [ ]:
#Despues filtro las peliculas que sean Argentinas y que tengan el genero Drama. Para eso utilizo un simple filter. Luego realizo un map de forma tal que los registros queden (movieId,movieTitulo) para luego realizar un join
moviesRdd = moviesRdd.filter(lambda x: "Argentina" in str(x.production_countries) and "Drama" in str(x.genres)).map(lambda x: (int(x.id),x.original_title))
moviesRdd.take(5)

[(124304, 'De eso no se habla'),
 (90148, 'Of Love and Shadows'),
 (123763, 'Yo, la peor de todas'),
 (34838, 'The Tango Lesson'),
 (65749, 'Tango, no me dejes nunca')]

In [ ]:
#Realizo un join entre moviesRdd y ratingsRdd que comparten la clave movieId. Los registros se muestran como (idMovie,(movieTitlo,cantRatings))
moviesRdd.join(ratingsRdd).takeOrdered(10, key=lambda x: -x[1][1])

[(1653, ('Diarios de motocicleta', 23335)),
 (16, ('Dancer in the Dark', 20517)),
 (45722, ('No mires para abajo', 15079)),
 (1956, ('Gerry', 4707)),
 (6636, ('El asaltante', 974)),
 (5333, ('Apartment Zero', 385)),
 (48596, ('Ana y los otros', 273)),
 (47261, ('Carancho', 234))]